In [ ]:
from graphviz import Digraph
import copy

In [ ]:
field = ["_", "X", "_", "_", "_", "_", "_", "_", "_"]
file_name = "edge"

def rotate_field(field):
    field_copy = copy.deepcopy(field)
    field_copy[0] = field[6]
    field_copy[1] = field[3]
    field_copy[2] = field[0]
    field_copy[3] = field[7]
    field_copy[4] = field[4]
    field_copy[5] = field[1]
    field_copy[6] = field[8]
    field_copy[7] = field[5]
    field_copy[8] = field[2]
    return field_copy

def mirrorVertical_field(field):
    field_copy = copy.deepcopy(field)
    field_copy[0] = field[2]
    field_copy[2] = field[0]
    field_copy[3] = field[5]
    field_copy[5] = field[3]
    field_copy[6] = field[8]
    field_copy[8] = field[6]
    return field_copy

def mirrorHorizontal_field(field):
    field_copy = copy.deepcopy(field)
    field_copy[0] = field[6]
    field_copy[1] = field[7]
    field_copy[2] = field[8]
    field_copy[6] = field[0]
    field_copy[7] = field[1]
    field_copy[8] = field[2]
    return field_copy

def field_to_text(field):
    txt = ""
    for idx, element in enumerate(field):
        txt += element + " "
        if (idx + 1) % 3 == 0:
            txt += "\n"
    return txt

lines = [
    [0, 1, 2],
    [3, 4, 5],
    [6, 7, 8],
    [0, 3, 6],
    [1, 4, 7],
    [2, 5, 8],
    [0, 4, 8],
    [2, 4, 6],
]

def is_game_over(field):
    game_over = False
    for line in lines:
        if (
            field[line[0]] != "_"
            and field[line[0]] == field[line[1]]
            and field[line[0]] == field[line[2]]
        ):
            game_over = True
            break
    return game_over

txt_empty = field_to_text(field)

In [34]:
file_format = "svg"

for dot, dot_type in [
    (Digraph("flow", format=file_format), "normal"),
    (Digraph("flow", engine="neato", format=file_format), "neato"),
    (Digraph("flow", engine="twopi", format=file_format), "twopi"),
]:
    dot.attr(rankdir="TB", fontsize="10", overlap="false", splines="true")
    dot.attr("node", shape="rectangle", fontname="consolas")

    edges = set()
    fields = set()
    non_dup_fields = set()
    counter = 0

    def do_all_fields(field, player="X", parent=txt_empty):
        global counter
        global edges
        global fields
        global non_dup_fields

        nextPlayer = "X"
        if player == nextPlayer:
            nextPlayer = "O"
        for i in range(len(field)):
            field_copy = copy.deepcopy(field)
            if field_copy[i] == "_":
                field_copy[i] = player
                game_over = is_game_over(field_copy)
                txt = field_to_text(field_copy)

                rotate1field = rotate_field(field_copy)
                rotate2field = rotate_field(rotate1field)
                rotate3field = rotate_field(rotate2field)

                mirrorVfield = mirrorVertical_field(field_copy)
                mirrorVHfield = mirrorHorizontal_field(mirrorVfield)
                mirrorVHVfield = mirrorVertical_field(mirrorVHfield)

                if txt not in fields:
                    if game_over:
                        dot.node(txt, fillcolor='#0f0', style='filled')
                    elif '_' not in txt:
                        dot.node(txt, fillcolor='#f00', style='filled')
                    else:
                        dot.node(txt)
                    fields.add(txt)
                    non_dup_fields.add(txt)
                    fields.add(field_to_text(rotate1field))
                    fields.add(field_to_text(rotate2field))
                    fields.add(field_to_text(rotate3field))
                    fields.add(field_to_text(mirrorVfield))
                    fields.add(field_to_text(mirrorVHfield))
                    fields.add(field_to_text(mirrorVHVfield))

                try:
                    check_edge = parent+txt
                except:
                    check_edge = ""
                if parent and check_edge not in edges and txt in non_dup_fields:
                    dot.edge(parent, txt)

                edges.add(check_edge)

                # dot.render("flowchart_normal_svg", cleanup=True)
                # dot.render(f"out_normal_svg/flowchart{counter:04d}", cleanup=True)
                counter +=1

                if not game_over:
                    do_all_fields(field_copy, nextPlayer, txt)

    do_all_fields(field, "O")   
    dot.render(f"20260105_flowchart_{dot_type}_{file_name}", cleanup=True)  # produces flowchart.svg
